In [ ]:
def get_page(page_number: int, job_name: str) -> BeautifulSoup:
    """
    Pobiera dane ze stwony nofluff jobs.

    @param page_number: numer strony, który chcemy pobrać
    @param job_name: nazwa stanowiska z którego pobieramy dane
    """
    url = 'https://nofluffjobs.com'  # szablon url

    # parametry przekazywane do request'a
    params =  {'criteria': job_name, 'page': page_number}  
    r = requests.get(url, params)

    return BeautifulSoup(r.text)


def parse_salary(salary) -> dict:
    """
    Parsuje informacje dotyczące wynagrodzenia w formacie low_value - high_value

    @param salary: kwota wynagrodzenia dostępna dla oferty
    @returns dict {'low', 'high', 'currency'}: słownik z informacją o dolnej/górnej stawce oraz walucie
    """
    def extended_parser(value: str) -> int:
        """
        Rozszerzony parser dla ofert z wartościami w milionach
        """
        factor = value[-1]
        factor = 10**3 if factor == 'k' else 10**6

        val = float(value[:-1])
        return val * factor        


    d = salary.split()
    if len(d) == 3:
        low_bound = int(d[0] + d[1])
        high_bound = low_bound
        currency = d[2]
    elif len(d) == 4:
        low_bound = extended_parser(d[0])
        high_bound = extended_parser(d[2])
        currency = d[3]
    else:
        low_bound = int(d[0] + d[1])
        high_bound = int(d[3] + d[4])
        currency = d[5]

    return {'low': low_bound,
            'high': high_bound,
            'currency': currency}


def parse_city(city) -> dict:
    if re.search('(Zdalna)', city):
         _city = "Zdalna"
         country = 'N/A'
    else:       
        (_city, country) = city.split(',')
        country = re.sub('\\n +.*', '', country).strip()


    return {'city': _city, 'country': country}


def parse_job(job) -> dict:
    job_info = job.find('div', class_='posting-info position-relative d-none d-lg-flex flex-grow-1').find_all('span')
    
    salary = job_info[0].text.strip()
    salary_data = parse_salary(salary)
    
    location = job.find('nfj-posting-item-city')
    if location is None:
        location = job_info[1]

    location = location.text.strip()
    location = parse_city(location)

    name = job.find('h3', class_='posting-title__position').text.strip()
    company = job.find('span', class_='posting-title__company').text.replace('w', '').strip()

    technology = job.find('a', class_='btn btn-outline-secondary btn-sm text-truncate')
    if technology:
        technology = technology.text.strip()
    else:
        technology = 'N/A'

    return {
        'location': location,
        'salary': salary_data,
        'name': name,
        'company': company,
        'technology': technology
    }    


def parse_jobs(jobs) -> list:
    results = []
    
    for job in jobs:
        results.append(parse_job(job))

    return results    


def is_page_empty(bs) -> bool:
    empty_condition = bs.find('h2', class_='text-white font-weight-bold')

    if empty_condition is None:
        return False
    else:
        return True


def get_data(job_name: str):
    page_number = 1
    results = []

    while True:
        print('Downloading data from page {}'.format(page_number))
        page = get_page(page_number, job_name)
        
        if is_page_empty(page):
            break

        jobs = [x.parent for x in page.find_all('div', class_='posting-image')]
        page_data = parse_jobs(jobs)
        results += page_data

        page_number += 1
        time.sleep(1)

    return results
